In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from glob import glob 
import sys

# combine data

In [2]:
def combine_data(purpose):
    if purpose == 'public':
        custinfo = pd.read_csv('./data/public_train_x_custinfo_full_hashed.csv')
        cdtx = pd.read_csv('./data/public_train_x_cdtx0001_full_hashed.csv')
        dp = pd.read_csv('./data/public_train_x_dp_full_hashed.csv')
        remit = pd.read_csv('./data/public_train_x_remit1_full_hashed.csv')
        alert_pub = pd.read_csv('./data/public_x_alert_date.csv')
        alert_train = pd.read_csv('./data/train_x_alert_date.csv')
        alert =pd.concat([alert_pub, alert_train])
        #label = pd.read_csv('./train_y_answer.csv')
        
    elif purpose == 'private':
        custinfo = pd.read_csv('./data/private_x_custinfo_full_hashed.csv')
        cdtx = pd.read_csv('./data/private_x_cdtx0001_full_hashed.csv')
        dp = pd.read_csv('./data/private_x_dp_full_hashed.csv')
        remit = pd.read_csv('./data/private_x_remit1_full_hashed.csv')
        alert_pub = pd.read_csv('./data/private_x_alert_date.csv')
        alert_train = pd.read_csv('./data/private_x_alert_date.csv')
        alert =pd.concat([alert_pub, alert_train])
        
    custs = custinfo.cust_id.unique()
    
    def process_date(x):
        dates = x[['date', 'tx_date', 'trans_date']].values
        u_dates = pd.unique(dates)
        u_dates = u_dates[~pd.isna(u_dates)]
        #print(u_dates)
        x['date'] = u_dates[0]
        return x
    
    all_data = []
    n_aks = 0
    missed = []
    for cust in tqdm(custs):
        dp_c = dp[dp.cust_id == cust]
        cdtx_c = cdtx[cdtx.cust_id == cust]
        remit_c = remit[remit.cust_id == cust]
        cust_data = pd.concat([cdtx_c, dp_c, remit_c], axis = 0)
        if len(cust_data) > 0:
            cust_data = cust_data.apply(lambda x: process_date(x), axis=1)
            cust_data.drop(columns = ['tx_date', 'trans_date'], inplace = True)
            #cust_data.fillna(-1, inplace = True)
            cust_data.sort_values(['date'], inplace = True)
            cust_data['alert_key'] = np.nan
            aks = custinfo[custinfo.cust_id == cust].alert_key # alert keys
            ads = alert[alert.alert_key.isin(aks)] # alert key-date mapping
            if len(ads) > 0:
                ads.sort_values(by=['date'], ascending=False, inplace=True)
                ads.drop_duplicates(inplace=True)
                n_aks += len(ads)
                for k, d in ads.values:
                    idx = cust_data[cust_data.date<=d].index
                    if len(idx) > 0:
                        cust_data.loc[idx, 'alert_key'] = k   
                    else:
                        missed.append([cust, k, d])
            #cust_data['alert_key'].fillna(method='bfill', inplace=True)
            cust_data.dropna(subset=['alert_key'], inplace=True)
            all_data.append(cust_data)
    
    all_data = pd.concat(all_data)
    
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    all_data.loc[:,'debit_credit'] = le.fit_transform(all_data.debit_credit)
    print()
    all_data.to_csv(f'./result/{purpose}_combined_data.csv')

In [3]:
combine_data('private')

  0%|          | 0/747 [00:00<?, ?it/s]/tmp/ipykernel_17682/487641858.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ads.sort_values(by=['date'], ascending=False, inplace=True)
/tmp/ipykernel_17682/487641858.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ads.drop_duplicates(inplace=True)
  0%|          | 1/747 [00:00<01:16,  9.78it/s]/tmp/ipykernel_17682/487641858.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ads.sor

# dp feature

In [24]:
combine = pd.read_csv ('./result/private_combined_data.csv')
df = combine.copy()
df.drop("Unnamed: 0",axis=1, inplace=True)
df.fillna(0, inplace=True)

In [25]:
new_data = {'cust_id' : [], 'alert_key' : [], 'transnation_rate' : [], 'total_amount_money_in':[], 'total_amount_money_out':[], 'var_money_in':[], 'var_money_out':[]}

cust_id_list = list(set(df["cust_id"].values.tolist()))

for i in range(len(cust_id_list)):
    df_temp = df[(df['cust_id'] == cust_id_list[i])]
    alert_key_list = list(set(df_temp["alert_key"].values.tolist()))
    for j in range(len(alert_key_list)):
        df_alert_key = df_temp[(df_temp['alert_key'] == alert_key_list[j])]

        df_internation = df_alert_key[~(df_alert_key['exchg_rate'] == 1)]

        internation_num = len(df_internation)
        
        sum_1 = df_internation[(df_internation['debit_credit']==1)]['tx_amt'].sum()
        sum_0 = df_internation[(df_internation['debit_credit']==0)]['tx_amt'].sum()

        var_1 = df_internation[(df_internation['debit_credit']==1)]['tx_amt'].std()
        var_0 = df_internation[(df_internation['debit_credit']==0)]['tx_amt'].std()

        new_data['cust_id'].append(cust_id_list[i])
        new_data['alert_key'].append(alert_key_list[j])
        new_data['transnation_rate'].append(internation_num)
        new_data['total_amount_money_in'].append(sum_1) 
        new_data['total_amount_money_out'].append(sum_0) 
        new_data['var_money_in'].append(var_1)
        new_data['var_money_out'].append(var_0)
        
feature_0 = pd.DataFrame(new_data)
feature_0.fillna(0, inplace=True)

In [26]:
df = combine.copy()   
df['day'] = 1        
feature = pd.DataFrame()

zero = df.copy()
zero.loc[((zero['debit_credit'] == 1) | (zero['debit_credit'] == 2)), 'tx_amt'] = 0
one = df.copy()
one.loc[((zero['debit_credit'] == 0) | (zero['debit_credit'] == 2)), 'tx_amt'] = 0
feature['0_sum'] = zero.groupby(['cust_id','alert_key'])['tx_amt'].sum()
feature['1_sum'] = one.groupby(['cust_id','alert_key'])['tx_amt'].sum()

feature['0_var'] = zero.groupby(['cust_id','alert_key'])['tx_amt'].std()
feature['1_var'] = one.groupby(['cust_id','alert_key'])['tx_amt'].std()
feature['day'] = feature['cross_bank_time'] = df.groupby(['cust_id','alert_key'])['day'].sum()
feature['cross_bank_time'] = df.groupby(['cust_id','alert_key'])['cross_bank'].sum()

# cddtx & remit feature

In [27]:
z = pd.DataFrame()
combine['day'] = 1
combine.loc[combine['cur_type'] != 47, 'd_cur_type'] = 1
z['amt_sum'] = combine.groupby(['cust_id','alert_key'])['amt'].sum()
z['amt_std'] = combine.groupby(['cust_id','alert_key'])['amt'].std()

z['d_cur_type'] = combine.groupby(['cust_id','alert_key'])['d_cur_type'].sum()
z['day'] = combine.groupby(['cust_id','alert_key'])['day'].sum()
z['trade_amount_usd'] = combine.groupby(['cust_id','alert_key'])['trade_amount_usd'].max()
comb_d = combine[['cust_id','alert_key','trade_amount_usd','trans_no']]
comb_d.set_index(['cust_id','alert_key','trade_amount_usd'],inplace = True)
z.reset_index(inplace = True)

z.set_index(['cust_id','alert_key','trade_amount_usd'],inplace = True)

temp_z = z.join(comb_d,how = 'outer')
temp_z['trans_no'].fillna(-1,inplace = True)
temp_z.reset_index(inplace = True)
z['trans_no_final'] = temp_z.groupby(['cust_id','alert_key'])['trans_no'].max(['trans_no'].count)
z.reset_index(inplace = True)

z['d_cur_type_ratio'] = z['d_cur_type']/z['day']
z['trade_amount_usd'].fillna(0,inplace = True)

z.fillna(0,inplace = True)
z.set_index(['cust_id','alert_key'],inplace = True)


# cdtx & remit + dp(no timeseries)

In [28]:
feature.drop(columns = 'day', inplace = True)
out = z.join(feature,how = 'outer')
print(out.shape)

out.reset_index(inplace = True)
out.set_index('alert_key', inplace = True)
print(out.shape)
#cdtx & remit + dp(no timeseries) + answer
f_out = out
feature_0.set_index(['cust_id','alert_key'], inplace = True)

feature = f_out.join(feature_0,how = 'outer')
feature['cross_bank_rate'] = feature['cross_bank_time'] / feature['day']
feature.drop(columns = ['cross_bank_time'], inplace = True) #'Unnamed: 0',


cust_b = pd.read_csv("./data/private_x_custinfo_full_hashed.csv")


(1835, 12)
(1835, 13)


# ccba feature

In [29]:

ccba_data = pd.read_csv("./data/public_train_x_ccba_full_hashed.csv")
ccba_data['use_ratio']=ccba_data['usgam']/ccba_data['cycam']
column_list = ['lupay', 'clamt', 'csamt','inamt', 'cucsm', 'cucah']
ccba = pd.DataFrame()

for i in column_list:
    ccba[ i + '_max'] = ccba_data.groupby(['cust_id'])[i].max()
    ccba[ i + '_std'] = ccba_data.groupby(['cust_id'])[i].std()



ccba['cycam_type']= ccba_data.groupby(['cust_id'])['cycam'].unique()
ccba['cytocam_times'] = ccba['cycam_type'].apply(lambda x: len(x))
ccba['cytocam_first'] = ccba['cycam_type'].apply(lambda x: x[0])
ccba['cytocam_last'] = ccba['cycam_type'].apply(lambda x: x[-1])


ccba['cytocam_variance'] = (ccba['cytocam_last'] - ccba['cytocam_first'])/ccba['cytocam_first']
ccba['cytocam_variance'].fillna(0,inplace = True)
ccba['cytocam_variance'].replace(np.inf, 1, inplace = True)



    
ccba.drop(columns = 'cycam_type', inplace = True)
ccba.reset_index(inplace = True)


In [23]:
feature.fillna(-5,inplace = True)
feature.rename(columns = {'0_sum':'sum_0',
                        '1_sum':'sum_1',
                        '0_var':'var_0',
                        '1_var':'var_1'}, inplace = True)

cust_b.set_index(['cust_id','alert_key'], inplace = True)
data = feature.join(cust_b, how = 'outer')

data['trans_no_final'].replace({-1: 5 },inplace = True)

ccba.set_index(['cust_id'], inplace = True)

data.drop(columns = ['cust_id'], inplace = True)
data.reset_index(inplace = True)
data.set_index(['cust_id'], inplace = True)
data = data.join(ccba, how = 'outer')

data.fillna(0,inplace = True)
data.reset_index(inplace = True)


data.to_csv('./result/featuresV1_Private.csv')